# Model tuning and selection
>  In this last chapter, you'll apply what you've learned to create a model that predicts which flights will be delayed.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 4 exercises "Introduction to PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [1]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

## What is logistic regression?

<div class=""><p>The model you'll be fitting in this chapter is called a <em>logistic regression</em>. This model is very similar to a linear regression, but instead of predicting a numeric variable, it predicts the probability (between 0 and 1) of an event.</p>
<p>To use this as a classification algorithm, all you have to do is assign a cutoff point to these probabilities. If the predicted probability is above the cutoff point, you classify that observation as a 'yes' (in this case, the flight being late), if it's below, you classify it as a 'no'!</p>
<p>You'll tune this model by testing different values for several <em>hyperparameters</em>. A <em>hyperparameter</em> is just a value in the model that's not estimated from the data, but rather is supplied by the user to maximize performance. For this course it's not necessary to understand the mathematics behind all of these values - what's important is that you'll try out a few different choices and pick the best one.</p>
<hr>
<p>Why do you supply hyperparameters?</p></div>

<pre>
Possible Answers
They explain information about the data.
<b>They improve model performance.</b>
They improve model fitting speed.
</pre>

**You supply hyperparameters to optimize your model.**

## Create the modeler

<p>The <code>Estimator</code> you'll be using is a <code>LogisticRegression</code> from the <code>pyspark.ml.classification</code> submodule.</p>

In [4]:
spark = (SparkSession.builder.appName("flights").getOrCreate())

Instructions
<ul>
<li>Import the <code>LogisticRegression</code> class from <code>pyspark.ml.classification</code>.</li>
<li>Create a <code>LogisticRegression</code> called <code>lr</code> by calling <code>LogisticRegression()</code> with no arguments.</li>
</ul>

In [5]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

**That's the first step to any modeling in PySpark.**

## Cross validation

<div class=""><p>In the next few exercises you'll be tuning your logistic regression model using a procedure called <em>k-fold cross validation</em>. This is a method of estimating the model's performance on unseen data (like your <code>test</code> DataFrame).</p>
<p>It works by splitting the training data into a few different partitions. The exact number is up to you, but in this course you'll be using PySpark's default value of three. Once the data is split up, one of the partitions is set aside, and the model is fit to the others. Then the error is measured against the held out partition. This is repeated for each of the partitions, so that every block of data is held out and used as a test set exactly once. Then the error on each of the partitions is averaged. This is called the <em>cross validation error</em> of the model, and is a good estimate of the actual error on the held out data.</p>
<p>You'll be using cross validation to choose the hyperparameters by creating a grid of the possible pairs of values for the two hyperparameters, <code>elasticNetParam</code> and <code>regParam</code>, and using the cross validation error to compare all the different models so you can choose the best one!</p>
<p>What does cross validation allow you to estimate?</p>
<hr></div>

<pre>
Possible Answers
<b>The model's error on held out data.</b>
The model's error on data used for fitting.
The time it will take to fit the model.
</pre>

**The cross validation error is an estimate of the model's error on the test set.**

## Create the evaluator

<div class=""><p>The first thing you need when doing cross validation for model selection is a way to compare different models. Luckily, the <code>pyspark.ml.evaluation</code> submodule has classes for evaluating different kinds of models. Your model is a binary classification model, so you'll be using the <code>BinaryClassificationEvaluator</code> from the <code>pyspark.ml.evaluation</code> module.</p>
<p>This evaluator calculates the area under the ROC. This is a metric that combines the two kinds of errors a binary classifier can make (false positives and false negatives) into a simple number. You'll learn more about this towards the end of the chapter!</p></div>

Instructions
<ul>
<li>Import the submodule <code>pyspark.ml.evaluation</code> as <code>evals</code>.</li>
<li>Create <code>evaluator</code> by calling <code>evals.BinaryClassificationEvaluator()</code> with the argument <code>metricName="areaUnderROC"</code>.</li>
</ul>

In [6]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

**Now you can compare models using the metric output by your evaluator!**

## Make a grid

<div class=""><p>Next, you need to create a grid of values to search over when looking for the optimal hyperparameters. The submodule <code>pyspark.ml.tuning</code> includes a class called <code>ParamGridBuilder</code> that does just that (maybe you're starting to notice a pattern here; PySpark has a submodule for just about everything!).</p>
<p>You'll need to use the <code>.addGrid()</code> and <code>.build()</code> methods to create a grid that you can use for cross validation. The <code>.addGrid()</code> method takes a model parameter (an attribute of the model <code>Estimator</code>, <code>lr</code>, that you created a few exercises ago) and a list of values that you want to try.
The <code>.build()</code> method takes no arguments, it just returns the grid that you'll use later.</p></div>

Instructions
<ul>
<li>Import the submodule <code>pyspark.ml.tuning</code> under the alias <code>tune</code>.</li>
<li>Call the class constructor <code>ParamGridBuilder()</code> with no arguments. Save this as <code>grid</code>.</li>
<li>Call the <code>.addGrid()</code> method on <code>grid</code> with <code>lr.regParam</code> as the first argument and <code>np.arange(0, .1, .01)</code> as the second argument. This second call is a function from the <code>numpy</code> module (imported <code>as np</code>) that creates a list of numbers from 0 to .1, incrementing by .01. Overwrite <code>grid</code> with the result.</li>
<li>Update <code>grid</code> again by calling the <code>.addGrid()</code> method a second time create a grid for <code>lr.elasticNetParam</code> that includes only the values <code>[0, 1]</code>.</li>
<li>Call the <code>.build()</code> method on <code>grid</code> and overwrite it with the output.</li>
</ul>

In [7]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid = grid.build()

**That's the last ingredient in your cross validation recipe!**

## Make the validator

<div class=""><p>The submodule <code>pyspark.ml.tuning</code> also has a class called <code>CrossValidator</code> for performing cross validation. This <code>Estimator</code> takes the modeler you want to fit, the grid of hyperparameters you created, and the evaluator you want to use to compare your models.</p>
<p>The submodule <code>pyspark.ml.tune</code> has already been imported as <code>tune</code>. You'll create the <code>CrossValidator</code> by passing it the logistic regression <code>Estimator</code> <code>lr</code>, the parameter <code>grid</code>, and the <code>evaluator</code> you created in the previous exercises.</p></div>

Instructions
<ul>
<li>Create a <code>CrossValidator</code> by calling <code>tune.CrossValidator()</code> with the arguments:<ul>
<li><code>estimator=lr</code></li>
<li><code>estimatorParamMaps=grid</code></li>
<li><code>evaluator=evaluator</code></li></ul></li>
<li>Name this object <code>cv</code>.</li>
</ul>

In [10]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)

## Fit the model(s)

<div class=""><p>You're finally ready to fit the models and select the best one!</p>
<p>Unfortunately, cross validation is a very computationally intensive procedure. Fitting all the models would take too long on DataCamp.</p>
<p>To do this locally you would use the code:</p>
<pre><code># Fit cross validation models
models = cv.fit(training)

Extract the best model
best_lr = models.bestModel
</code></pre>
<p>Remember, the training data is called <code>training</code> and you're using <code>lr</code> to fit a logistic regression model. Cross validation selected the parameter values <code>regParam=0</code> and <code>elasticNetParam=0</code> as being the best. These are the default values, so you don't need to do anything else with <code>lr</code> before fitting the model.</p></div>

In [46]:
%%capture
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/flights_small.csv
spark = (SparkSession.builder.appName("flights").getOrCreate())
flights = (spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("flights_small.csv"))
flights.createOrReplaceTempView("flights")

!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/planes.csv
planes = (spark.read.format("csv").option("inferSchema", "true").option("header", "true").load('planes.csv'))
planes.createOrReplaceTempView("planes")

planes = planes.withColumnRenamed('year', 'plane_year')
model_data = flights.join(planes, on='tailnum', how='leftouter')

model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn('air_time', model_data.air_time.cast('integer'))
model_data = model_data.withColumn('month', model_data.month.cast('integer'))
model_data = model_data.withColumn('plane_year', model_data.plane_year.cast('integer'))

model_data = model_data.withColumn('plane_age', model_data.year - model_data.plane_year)
model_data = model_data.withColumn('is_late', model_data.arr_delay > 0)
model_data = model_data.withColumn('label', model_data.is_late.cast('integer'))
model_data = model_data.filter('arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and plane_year is not NULL')

carr_indexer = StringIndexer(inputCol='carrier', outputCol='carrier_index')
carr_encoder = OneHotEncoder(inputCol='carrier_index', outputCol='carrier_fact')
dest_indexer = StringIndexer(inputCol='dest', outputCol='dest_index')
dest_encoder = OneHotEncoder(inputCol='dest_index', outputCol='dest_fact')

vec_assembler = VectorAssembler(inputCols=['month', 'air_time', 'carrier_fact', 'dest_fact', 'plane_age'], outputCol='features')
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

piped_data = flights_pipe.fit(model_data).transform(model_data)
piped_data = piped_data.drop("type").drop("manufacturer").drop("manufacturer").drop('model').drop('engine').drop('engines').drop('seats').drop('speed')

training, test = piped_data.randomSplit([.6, .4])

Instructions
<ul>
<li>Create <code>best_lr</code> by calling <code>lr.fit()</code> on the <code>training</code> data.</li>
<li>Print <code>best_lr</code> to verify that it's an object of the <code>LogisticRegressionModel</code> class.</li>
</ul>

In [47]:
# Call lr.fit()
best_lr = lr.fit(training)

# Print best_lr
print(best_lr)

LogisticRegressionModel: uid=LogisticRegression_8b069011e85a, numClasses=2, numFeatures=81


## Evaluating binary classifiers

For this course we'll be using a common metric for binary classification algorithms call the AUC, or area under the curve. In this case, the curve is the ROC, or receiver operating curve. The details of what these things actually measure isn't important for this course. All you need to know is that for our purposes, the closer the AUC is to one (1), the better the model is!

If you've created a perfect binary classification model, what would the AUC be?

<pre>
Possible Answers
-1
<b>1</b>
0
.5
</pre>

**An AUC of one represents a model that always perfectly classifies observations.**

## Evaluate the model

<p>Remember the test data that you set aside waaaaaay back in chapter 3? It's finally time to test your model on it! You can use the same evaluator you made to fit the model.</p>

Instructions
<ul>
<li>Use your model to generate predictions by applying <code>best_lr.transform()</code> to the <code>test</code> data. Save this as <code>test_results</code>.</li>
<li>Call <code>evaluator.evaluate()</code> on <code>test_results</code> to compute the AUC. Print the output.</li>
</ul>

In [48]:
# Use the model to predict the test set
test_results = best_lr.transform(test)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.6911151128784382


**What do you think of the AUC? Your model isn't half bad! You went from knowing nothing about Spark to doing advanced machine learning. Great job on making it to the end of the course! The next steps are learning how to create large scale Spark clusters and manage and submit jobs so that you can use models in the real world. Check out some of the other DataCamp courses that use Spark! And remember, Spark is still being actively developed, so there's new features coming all the time!**